In [ ]:
import random
import gym
import numpy as np
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

EPISODES = 1000

class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=100000)
        self.gamma = 0.95    # discount rate
        self.epsilon = 1.0 # exploration rate
        self.epsilon2 = 0.95
        self.epsilon_min = 0.1
        self.epsilon_decay = 0.9999
        self.learning_rate = 0.05
        self.model = self._build_model()

    def _build_model(self):
        # Neural Net for Deep-Q learning Model
        model = Sequential()
        model.add(Dense(85, input_dim=self.state_size, activation='relu'))
        model.add(Dense(45, activation='relu'))
        model.add(Dense(25, activation='relu'))
        model.add(Dense(10, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse',
                      optimizer=Adam(lr=self.learning_rate))
        return model

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            if np.random.rand() <= self.epsilon2:
                return get_action(state)
            else:
                return random.randrange(self.action_size) 
        #print( state.shape)
        act_values = self.model.predict(np.expand_dims(state, axis=0))
        return np.argmax(act_values[0])  # returns action

    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target = (reward + self.gamma *
                          np.amax(self.model.predict(np.expand_dims(next_state, axis=0))[0]))
            target_f = self.model.predict(np.expand_dims(state, axis=0))
            target_f[0][action] = target
            self.model.fit(np.expand_dims(state, axis=0), target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def load(self, name):
        self.model.load_weights(name)

    def save(self, name):
        self.model.save_weights(name)


env = gym.make('SpaceInvaders-v4')
state_size = 3021
action_size = env.action_space.n
agent = DQNAgent(state_size, action_size)
# agent.load("./save/cartpole-dqn.h5")
done = False
batch_size = 128

for e in range(EPISODES):
    state = env.reset()
    actual_reward = 0
    state = load_transform(state)
    for time in range(3000):
        #env.render()
        action = agent.act(state)
        next_state, reward, done, _ = env.step(action)
        actual_reward += reward
        reward = reward if not done else -10
        next_state = load_transform(next_state)
        agent.remember(state, action, reward, next_state, done)
        state = next_state
        if done:
            break
        if len(agent.memory) > batch_size:
            agent.replay(batch_size)
    print("episode: {}/{}, score: {}, reward: {}, e: {:.2} \r"
          .format(e, EPISODES, time, actual_reward, agent.epsilon))


/home/andres/thesis_project/keras2/lib/python3.6/site-packages/skimage/transform/_warps.py:24: UserWarning: The default multichannel argument (None) is deprecated.  Please specify either True or False explicitly.  multichannel will default to False starting with release 0.16.
  warn('The default multichannel argument (None) is deprecated.  Please '
/home/andres/thesis_project/keras2/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


episode: 0/1000, score: 600, reward: 55.0, e: 0.95 
episode: 1/1000, score: 372, reward: 15.0, e: 0.92 
episode: 2/1000, score: 591, reward: 295.0, e: 0.87 
episode: 3/1000, score: 521, reward: 130.0, e: 0.82 
episode: 4/1000, score: 657, reward: 130.0, e: 0.77 
episode: 5/1000, score: 531, reward: 65.0, e: 0.73 
episode: 6/1000, score: 650, reward: 105.0, e: 0.68 
episode: 7/1000, score: 761, reward: 270.0, e: 0.63 
episode: 8/1000, score: 424, reward: 120.0, e: 0.61 
episode: 9/1000, score: 383, reward: 75.0, e: 0.58 
episode: 10/1000, score: 490, reward: 80.0, e: 0.56 
episode: 11/1000, score: 881, reward: 235.0, e: 0.51 
episode: 12/1000, score: 1048, reward: 550.0, e: 0.46 
episode: 13/1000, score: 399, reward: 55.0, e: 0.44 
episode: 14/1000, score: 682, reward: 140.0, e: 0.41 
episode: 15/1000, score: 425, reward: 105.0, e: 0.4 
episode: 16/1000, score: 429, reward: 105.0, e: 0.38 
episode: 17/1000, score: 414, reward: 75.0, e: 0.36 
episode: 18/1000, score: 382, reward: 75.0, e

In [5]:
env = gym.make('Cart-v4')

In [1]:
import gym
env = gym.make('SpaceInvaders-v4')

In [12]:
env.unwrapped.get_action_meanings()

['NOOP', 'FIRE', 'RIGHT', 'LEFT', 'RIGHTFIRE', 'LEFTFIRE']

In [3]:
from keras.models import load_model
model = load_model('/media/andres/Baymax/predictor.h5')
encoder = load_model('/media/andres/Baymax/encoder100.h5')

Using TensorFlow backend.
/home/andres/thesis_project/keras2/lib/python3.6/site-packages/keras/engine/saving.py:269: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [5]:
env = gym.make('SpaceInvaders-v4')
state_size = env.observation_space.shape[0] #377
action_size = env.action_space.n
agent = DQNAgent(state_size, action_size)

NameError: name 'DQNAgent' is not defined

In [13]:
state = env.reset()

In [1]:
import matplotlib.pyplot as plt

from skimage import io, color
from skimage.transform import rescale
from skimage.draw import rectangle
from skimage.measure import label, regionprops
from skimage.filters import threshold_otsu
from skimage.morphology import closing, square
import h5py

def load_transform(img):
    img = img[25:195,0:160]
    img_gray = color.rgb2gray(img)
    
    thresh = threshold_otsu(img_gray)
    bw = closing(img_gray > thresh, square(3))
    cleared = bw
    label_image = label(cleared)
    for region in regionprops(label_image):
        if region.area >= 10:
            minr, minc, maxr, maxc = region.bbox
            rr, cc = rectangle(start=(minr, minc), end=(maxr, maxc), shape=cleared.shape)
            cleared[rr, cc] = 1

    image_rescaled = rescale(cleared, 1.0 / 3.0, anti_aliasing=False)
    
    return image_rescaled.reshape(3021)

In [33]:
import numpy as np

state_transformed = load_transform(state)
encoded = encoder.predict(np.expand_dims(state_transformed, axis=0))
prediction = model.predict(encoded)

/home/andres/thesis/keras/lib/python3.6/site-packages/skimage/transform/_warps.py:24: UserWarning: The default multichannel argument (None) is deprecated.  Please specify either True or False explicitly.  multichannel will default to False starting with release 0.16.
  warn('The default multichannel argument (None) is deprecated.  Please '
/home/andres/thesis/keras/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [36]:
prediction.argmax()

1

In [28]:
state_transformed.shape

(3021,)

In [1]:
actions = ['NOOP', 'FIRE', 'UP', 'RIGHT', 'LEFT', 'DOWN', 'UPRIGHT', 'UPLEFT', 'DOWNRIGHT', 'DOWNLEFT', 'UPFIRE', 'RIGHTFIRE', 'LEFTFIRE', 'DOWNFIRE', 'UPRIGHTFIRE', 'UPLEFTFIRE', 'DOWNRIGHTFIRE', 'DOWNLEFTFIRE']
env_actions  = ['NOOP', 'FIRE', 'RIGHT', 'LEFT', 'RIGHTFIRE', 'LEFTFIRE']

import matplotlib.pyplot as plt

from skimage import io, color
from skimage.transform import rescale
from skimage.draw import rectangle
from skimage.measure import label, regionprops
from skimage.filters import threshold_otsu
from skimage.morphology import closing, square
from keras.models import load_model
import h5py

model = load_model('/media/andres/Baymax/predictor.h5')
encoder = load_model('/media/andres/Baymax/encoder100.h5')

def load_transform(img):
    img = img[25:195,0:160]
    img_gray = color.rgb2gray(img)
    
    thresh = threshold_otsu(img_gray)
    bw = closing(img_gray > thresh, square(3))
    cleared = bw
    label_image = label(cleared)
    for region in regionprops(label_image):
        if region.area >= 10:
            minr, minc, maxr, maxc = region.bbox
            rr, cc = rectangle(start=(minr, minc), end=(maxr, maxc), shape=cleared.shape)
            cleared[rr, cc] = 1

    image_rescaled = rescale(cleared, 1.0 / 3.0, anti_aliasing=False)
    
    return image_rescaled.reshape(3021)

def get_action(state):
    encoded = encoder.predict(np.expand_dims(state, axis=0))
    prediction = model.predict(encoded)[0]
    prediction_index = prediction.argmax()
    #print(prediction_index)
    try: 
        return env_actions.index(actions[prediction_index])
    except:
        return 0
    

Using TensorFlow backend.
/home/andres/thesis_project/keras2/lib/python3.6/site-packages/keras/engine/saving.py:269: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [3]:
env_actions.index('FIRE')

1